### Problem Statement

- Determine how Pegasus schedules tasks to resources
- Determine metrics to compute (e.g. resource utilization and makespan)
- Determine how to simulate estimated runtimes of tasks with varying resource levels
    - Perform topological sort of DAG based on parent values with associate priority values (correlated)

### Data Cleaning

In [1]:
import os
import sys
import pandas as pd 

os.chdir(os.path.dirname("../pegasus-traces/"))
sys.path.append(os.path.dirname("../pegasus-traces/"))

In [2]:
!pwd

/Users/surya/Documents/sdm/envs/pegasus-traces


In [23]:
directory = 'cycles/chameleon-cloud/'
files = [os.path.join(directory, file) for file in os.listdir(directory) if os.path.isfile(os.path.join(directory, file))]
dfs = [pd.read_json(file) for file in files]
print(files[0])
display(dfs[0])

cycles/chameleon-cloud/cycles-chameleon-1l-3c-12p-001.json


,name,description,createdAt,schemaVersion,wms,author,workflow
url,Cycles-20200413T205119Z-0,Trace generated with wrench-pegasus-parser.py ...,2020-04-13T15:24:55.676052Z,1,http://pegasus.isi.edu,NaN,NaN
version,Cycles-20200413T205119Z-0,Trace generated with wrench-pegasus-parser.py ...,2020-04-13T15:24:55.676052Z,1,4.9.3,NaN,NaN
name,Cycles-20200413T205119Z-0,Trace generated with wrench-pegasus-parser.py ...,2020-04-13T15:24:55.676052Z,1,Pegasus,rafsilva,NaN
email,Cycles-20200413T205119Z-0,Trace generated with wrench-pegasus-parser.py ...,2020-04-13T15:24:55.676052Z,1,NaN,support@wrench-project.org,NaN
makespan,Cycles-20200413T205119Z-0,Trace generated with wrench-pegasus-parser.py ...,2020-04-13T15:24:55.676052Z,1,NaN,NaN,1544.0
jobs,Cycles-20200413T205119Z-0,Trace generated with wrench-pegasus-parser.py ...,2020-04-13T15:24:55.676052Z,1,NaN,NaN,"[{'name': 'baseline_cycles_ID0000001', 'type':..."
machines,Cycles-20200413T205119Z-0,Trace generated with wrench-pegasus-parser.py ...,2020-04-13T15:24:55.676052Z,1,NaN,NaN,"[{'nodeName': 'compute-5', 'system': 'linux', ..."
executedAt,Cycles-20200413T205119Z-0,Trace generated with wrench-pegasus-parser.py ...,2020-04-13T15:24:55.676052Z,1,NaN,NaN,20200413T205127+0000


In [22]:
display(dfs[0])
dfs = dfs[:1]

for df in dfs: 
    print('Resources')
    pd.set_option('display.max_columns', None)
    resources = pd.DataFrame(df.loc['machines']['workflow']) 
    display(resources)
    pd.reset_option('max_columns')
    
    jobs = df.loc['jobs']['workflow']
    jobs = pd.DataFrame(jobs) 
    print('Jobs')
    pd.set_option('display.max_rows', None)
    display(jobs)
    pd.reset_option('max_rows')

Resources


,nodeName,system,architecture,memory,release,cpu
0,compute-5,linux,x86_64,131795984,4.15.0-96-generic,"{'count': 48, 'vendor': 'GenuineIntel', 'speed..."
1,compute-7,linux,x86_64,131795984,4.15.0-96-generic,"{'count': 48, 'vendor': 'GenuineIntel', 'speed..."
2,compute-3,linux,x86_64,131795984,4.15.0-96-generic,"{'count': 48, 'vendor': 'GenuineIntel', 'speed..."
3,compute-4,linux,x86_64,131795984,4.15.0-96-generic,"{'count': 48, 'vendor': 'GenuineIntel', 'speed..."


Jobs


,name,type,runtime,parents,files,avgCPU,arguments,machine,priority
0,baseline_cycles_ID0000001,compute,19.663,[],"[{'link': 'input', 'name': 'pongo.soil', 'size...",11.1377,"[--baseline, True, --fertilizer-increase, Fals...",compute-4,20
1,cycles_ID0000002,compute,1.497,[baseline_cycles_ID0000001],"[{'link': 'input', 'name': 'pongo.soil', 'size...",97.2612,"[--baseline, False, --fertilizer-increase, Fal...",compute-7,30
2,fertilizer_increase_cycles_ID0000003,compute,1.530,[baseline_cycles_ID0000001],"[{'link': 'input', 'name': 'pongo.soil', 'size...",98.8889,"[--baseline, False, --fertilizer-increase, Tru...",compute-4,30
3,cycles_fertilizer_increase_output_parser_ID000...,compute,0.055,"[fertilizer_increase_cycles_ID0000003, cycles_...","[{'link': 'input', 'name': 'fertilizer_increas...",98.1818,"[--params-file, cycles_params-7ad729864a79597b...",compute-3,40
4,baseline_cycles_ID0000005,compute,49.433,[],"[{'link': 'input', 'name': 'pongo.soil', 'size...",5.7027,"[--baseline, True, --fertilizer-increase, Fals...",compute-5,20
5,cycles_ID0000006,compute,12.029,[baseline_cycles_ID0000005],"[{'link': 'input', 'name': 'pongo.soil', 'size...",16.4935,"[--baseline, False, --fertilizer-increase, Fal...",compute-7,30
6,fertilizer_increase_cycles_ID0000007,compute,11.977,[baseline_cycles_ID0000005],"[{'link': 'input', 'name': 'pongo.soil', 'size...",16.6486,"[--baseline, False, --fertilizer-increase, Tru...",compute-5,30
7,cycles_fertilizer_increase_output_parser_ID000...,compute,0.056,"[fertilizer_increase_cycles_ID0000007, cycles_...","[{'link': 'input', 'name': 'cycles_season-fc1a...",98.2143,"[--params-file, cycles_params-fc1a6b73d7246ddf...",compute-4,40
8,baseline_cycles_ID0000009,compute,54.283,[],"[{'link': 'input', 'name': 'pongo.soil', 'size...",6.0406,"[--baseline, True, --fertilizer-increase, Fals...",compute-5,20
9,cycles_ID0000010,compute,25.609,[baseline_cycles_ID0000009],"[{'link': 'input', 'name': 'pongo.soil', 'size...",7.6106,"[--baseline, False, --fertilizer-increase, Fal...",compute-4,30


### Gym Environment

#### MDP Formulation [In Progress]
- State Space
    - Pairs
        - Node (name, memory, cpus)
        - Task (name, memory required, cpus required)

- Action Space
    - Run Task A on Node B (incurs cost of time to run task)
        - Ordering of DAG adds constrains on feasible actions
    
- Cost Function
    - Processing Time to Complete Run
   
- Goal
    - Minimize cost
    
- Algorithm
    - PPO
    
    

### References
https://pegasus.isi.edu/documentation/examples/

https://cobweb.cs.uga.edu/~kochut/teaching/8350/Papers/Workflow/Pegasus.pdf

https://dl.acm.org/doi/abs/10.1016/j.engappai.2019.02.013

RL HPC Batch Scheduler
https://arxiv.org/abs/1910.08925

RL DAG Workflow Scheduler
https://www.sciencedirect.com/science/article/pii/S0952197619300351?via=ihub

Task Scheduling Based on Adaptive Priority Experience Replay on Cloud Platforms 
https://www.mdpi.com/2079-9292/12/6/1358